# NLP wiht LSTM TensorFlow Youtube Tutorial 

Video: https://www.youtube.com/watch?v=ZMudJXhsUpY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=6

Code: https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb#scrollTo=w9vH8Y59ajYL

## Setup 

In [83]:
import tensorflow as tf
import keras 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import json

import numpy as np 
import os
import random
import re

#os.environ["TF_USE_LEGACY_KERAS"] = "1" # https://github.com/tensorflow/tfjs/issues/8328#issuecomment-2212490809

! echo $TF_USE_LEGACY_KERAS
! python3 --version
print(f"Tensorflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")


Python 3.10.15
Tensorflow Version: 2.15.1
Keras Version: 2.15.0


##  Einstellungen 

In [84]:
traning_new_model = True   
save_wort_index = True              # Soll ein neues Modell traniert werden ?
convert_to_js_model = True
copie_js_model_to_react = True          # Soll das js Modell direkt rüber kopiert werden ?

example_text = "Hallo ich bin"
traning_sentences = 500
model_name = "lstm" 

text_file = os.path.abspath("/workspace/dataset/10k_sentences.txt")
project_dir = os.path.abspath("/workspace/models/hero_to_zero_lstm")
word_index_file = os.path.join(project_dir, 'tokenizer_word_index.json')

model_dir = os.path.join(project_dir, model_name)
js_model_dir = os.path.join(project_dir, model_name + "_js")

react_model_dir = os.path.join(os.path.abspath("/workspace/webpage/silvan-webpage/public/models"),  model_dir)

## Data

In [85]:
def lade_zufaellige_zeilen(dateipfad, traning_sentences):

    # Zeilen zählne
    with open(dateipfad, 'r') as datei:
        anzahl_zeilen = sum(1 for zeile in datei)
    print(f"Text beinhaltet {anzahl_zeilen} Zeile \n")
    
    # Zufällige Zeilennummern generieren
    zufaellige_zeilennummern = set(random.sample(range(anzahl_zeilen), traning_sentences))
    print(f"{anzahl_zeilen} zufällige Zahlen zwischen 1 und {anzahl_zeilen}: {zufaellige_zeilennummern} \n")
    
    # Ausgewählten Zeilen laden
    ausgewaehlte_zeilen = ""
    with open(dateipfad, 'r') as datei:
        for nummer, zeile in enumerate(datei):
            if nummer in zufaellige_zeilennummern:
                zeile = re.sub(r'[^a-zA-Zäöüß.,!? \n]', '', zeile)
                ausgewaehlte_zeilen += zeile.strip().lower() + "\n"
    print(f"Trainingstext: \n{ausgewaehlte_zeilen[:100]}")
    
    return ausgewaehlte_zeilen

# Beispiel-Aufruf der Funktion
data = lade_zufaellige_zeilen(text_file, traning_sentences)


Text beinhaltet 9902 Zeile 

9902 zufällige Zahlen zwischen 1 und 9902: {8193, 6151, 8203, 6164, 31, 6178, 8227, 36, 8232, 8233, 2090, 4140, 4141, 49, 2121, 4172, 79, 102, 4198, 8307, 115, 2167, 8315, 2177, 129, 2178, 135, 4234, 6285, 6286, 2191, 8336, 2194, 6302, 8353, 8355, 2213, 2214, 174, 2226, 8375, 8377, 6332, 6342, 2249, 6349, 8400, 4305, 2258, 213, 6358, 6368, 4321, 2283, 2284, 8433, 4342, 8443, 2300, 4352, 259, 6408, 270, 274, 2323, 2325, 4374, 279, 6423, 8474, 4378, 8476, 2337, 6437, 4397, 8495, 312, 8505, 6459, 8510, 8514, 4430, 2391, 4448, 2401, 357, 2406, 6503, 8550, 6502, 4464, 6520, 8571, 6529, 8599, 4504, 410, 6561, 417, 2469, 8618, 6575, 4528, 2481, 432, 8627, 2486, 6584, 2488, 442, 4540, 2493, 445, 2508, 2509, 464, 4561, 2516, 4564, 6614, 477, 2532, 4584, 2545, 6641, 4593, 4604, 2561, 6658, 8707, 8710, 2570, 2576, 2581, 2587, 8734, 2591, 2598, 6701, 8749, 8752, 563, 8770, 8771, 2635, 6740, 6749, 610, 6755, 2660, 2662, 6758, 614, 4716, 8813, 2668, 2675, 2679, 2686, 473

In [86]:
def erstelle_tokenizer(data, word_index_file, save_wort_index):

    # Korpus erstellen und Tokenizer initialisieren
    tokenizer = Tokenizer()
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    print(f"Corpus mit der länge {len(corpus)}: \n{corpus} \n")
    print(f"Der Tokenizer kennt {total_words} Wörter \n")
    print(f"Die Wörter werden folgenden Indexen zugeordnet: \n{tokenizer.word_index} \n")
   
    
    # Wort-Index-Mapping in JSON-Datei speichern
    if save_wort_index:
        print(f"Speichert word_index_file unter '{word_index_file}' ")
        os.makedirs(os.path.dirname(word_index_file), exist_ok=True)
        word_index = tokenizer.word_index
        with open(word_index_file, 'w') as f:
            json.dump(word_index, f)

    return corpus, tokenizer, total_words

corpus, tokenizer, total_words = erstelle_tokenizer(data, word_index_file, save_wort_index)


Corpus mit der länge 501: 
['achtung, limited edition!', 'afghanistan empörung über zivile opfer .', 'aktionäre werden in kürze eine einladung erhalten.', 'alle musikstile sind gefragt.', 'allerdings mit unterschiedlichem schwierigkeitsgrad.', 'allerdings war das geschäft in der rezession auch tief eingebrochen.', 'alle signalwege des pds sind voll digitalisiert.', 'alles wissenswerte über schweden.', 'also nach der ffnungsdauer fragen.', 'am freitagabend wurden die siegerehrungen vorgenommen und die pokale überreicht.', 'an den festnahmen waren etwa  polizisten beteiligt.', 'andere materialien können vor ort günstig eingekauft werden.', 'an der indexspitze legten bmw , prozent zu.', 'an der rezeption erbeuteten sie lediglich einen geringen geldbetrag.', 'angesichts des steigenden yen standen weiterhin exporttitel auf der abschussliste.', 'anstelle des druidenwalzers kann ein spieler auch den druidensprung machen.', 'auch das thema des nächsten filmwettbewerbs ist nun bekannt.', 'auch 

In [87]:
def generier_daten(corpus, tokenizer, total_words):

    input_sequences = []
    is_first_line = True

    # Wandelt jede Zeile in eine Liste von Wortindizes und generiert n-Gramm-Sequenzen
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        print(f"Satz: {line} tokenisiert zu: {token_list} \n") if is_first_line == True else None

        # n-Gramm-Sequenz generieren
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            print(f"Generierte n-Gramm-Sequenz: {n_gram_sequence}") if is_first_line == True else None
        
        is_first_line = False

    # Alle Sequences auf gleiche Länge bringen
    max_sequence_len = max([len(seq) for seq in input_sequences])
    print(f"\nLängster Satz: {max_sequence_len}")
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    print(f"Padding mit der Shape {input_sequences.shape} : {input_sequences[0]}")


    # Aufteilen in Prädiktoren (xs) und Labels (ys)
    xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
    ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

    return xs, ys, max_sequence_len

xs, ys, max_sequence_len = generier_daten(corpus, tokenizer, total_words)


Satz: achtung, limited edition! tokenisiert zu: [281, 282, 283] 

Generierte n-Gramm-Sequenz: [281, 282]
Generierte n-Gramm-Sequenz: [281, 282, 283]

Längster Satz: 13


Padding mit der Shape (2733, 13) : [  0   0   0   0   0   0   0   0   0   0   0 281 282]


## Modell

In [88]:
# Modell Struktur 

from tensorflow.keras.models import load_model

def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=total_words,                  # Anzahl aller Wörter
                        output_dim=100,                         # Umwandlung des Wortes (des Integers) in den einen n-dimensionanlen Vektor 
                        input_length=(max_sequence_len-1)))     # Länge der Eingaben (Anzahl Wörter)
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation='softmax'))
    
    # Optimizer und Modell-Compile
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Prüfen, ob das Modell bereits existiert
if os.path.exists(model_dir) and traning_new_model == False:
    print("Modell gefunden. Lade bestehendes Modell...")
    model = load_model(model_dir)  
    
else:
    if traning_new_model ==True:
        print("Neuse Model soll traniert werden")
    else: 
        print("Altes Modell nicht gefunden")
    
    print("Startet Training \n\n")
    model = create_model()  # Neues Modell erstellen
    
    # Modell trainieren
    history = model.fit(xs, ys, epochs=80, verbose=1)
    
    # Modell speichern
    model.save(model_dir)
    print(f"\nModell gespeichert unter: {model_dir}")


Neuse Model soll traniert werden
Startet Training 


Epoch 1/80
86/86 [==============================] - 9s 42ms/step - loss: 7.2290 - accuracy: 0.0333
Epoch 2/80
86/86 [==============================] - 4s 42ms/step - loss: 6.4074 - accuracy: 0.0340
Epoch 3/80
86/86 [==============================] - 3s 40ms/step - loss: 5.5860 - accuracy: 0.0640
Epoch 4/80
86/86 [==============================] - 3s 37ms/step - loss: 4.3308 - accuracy: 0.1475
Epoch 5/80
86/86 [==============================] - 3s 36ms/step - loss: 2.7847 - accuracy: 0.3893
Epoch 6/80
86/86 [==============================] - 3s 38ms/step - loss: 1.6060 - accuracy: 0.6469
Epoch 7/80
86/86 [==============================] - 4s 47ms/step - loss: 0.9545 - accuracy: 0.7896
Epoch 8/80
86/86 [==============================] - 4s 45ms/step - loss: 0.6835 - accuracy: 0.8467
Epoch 9/80
86/86 [==============================] - 4s 42ms/step - loss: 0.5850 - accuracy: 0.8639
Epoch 10/80
86/86 [==============================] - 3s 

INFO:tensorflow:Assets written to: /workspace/models/hero_to_zero_lstm/lstm/assets



Modell gespeichert unter: /workspace/models/hero_to_zero_lstm/lstm


In [89]:
# Zugriff auf Embedding-Schicht
embedding_layer = model.layers[0]  # Die erste Schicht ist die Embedding-Schicht

# Eingabedimensionen (Vokabulargröße) und Ausgabedimensionen (Embedding-Dimension)
embedding_input_dim = embedding_layer.input_dim                 # Anzahl der Wörter im Vokabular
embedding_input_shape = embedding_layer.input_shape             # Form der Eingabedaten (z.B. Batchsize, Satzlänge)
embedding_output_shape = embedding_layer.output_shape           # Form der Ausgabedaten (z.B. Batchsize, Satzlänge, Embedding-Dimension)

embedding_weights = embedding_layer.get_weights()               # Zugriff auf die Gewichte der Embedding-Schicht
embedding_output_dim = embedding_layer.output_dim               # Anzahl der Dimensionen für jedes Wort (z.B. 100 für 100-Dimensionales Embedding)


# Ausgabe der Informationen
print(f"Embedding Eingabedimensionen: {embedding_input_dim}")
print(f"Form der Eingabedaten: {embedding_input_shape}")  # Form der Eingabedaten
print(f"Form der Ausgabedaten: {embedding_output_shape}")  # Form der Ausgabedaten

print(f"Embedding Gewichtedimensionen: {embedding_weights[0].shape}")  # Gewichtedimensionen der Embedding-Matrix
print(f"Embedding Ausgabedimensionen: {embedding_output_dim}")


Embedding Eingabedimensionen: 1681
Form der Eingabedaten: (None, 12)
Form der Ausgabedaten: (None, 12, 100)
Embedding Gewichtedimensionen: (1681, 100)
Embedding Ausgabedimensionen: 100


## Verwendung 

### Text generieren

In [90]:
next_words = 10
seed_text = example_text

for _ in range(next_words):
    # Tokenisiere den aktuellen Seed-Text und führe Padding durch
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    
    # Mache eine Vorhersage basierend auf dem aktuellen Seed-Text
    predicted_probs = model.predict(token_list, verbose=0)[0]
    predicted = np.argmax(predicted_probs)  # Bestimme das Wort mit der höchsten Wahrscheinlichkeit
    
    # Finde das vorhergesagte Wort im Tokenizer-Wörterbuch
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    
    # Wenn das Wort gefunden wurde, erweitere den Seed-Text
    if output_word:
        seed_text += " " + output_word
    else:
        print("Kein Wort gefunden für den vorhergesagten Index.")
        break

# Ausgabe des vollständigen Textes
print("Erweiterter Text:", seed_text)


Erweiterter Text: Hallo ich bin doch kein mantafahrer auch für tourismus freizeit und velofahrer aufgewertet


### Nextes Word vorhesagen  

In [91]:
import numpy as np

top_k = 10  # Anzahl der Top-Wörter
seed_text = "Ich"

# Tokenisierung und Padding des Eingabetexts
token_list = tokenizer.texts_to_sequences([seed_text])[0]
print(f"Eingabe: '{seed_text}' tokeiziert zu: {token_list}")
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
print(f"Mit Paddig: {token_list}")

# Vorhersagen berechnen
predictions = model.predict(token_list, verbose=0)[0]  # Softmax-Ausgabe

# Top-10 wahrscheinlichste Wörter finden
top_indices = np.argsort(predictions)[-top_k:][::-1]  #Sortiere absteigend

# Wörter und Wahrscheinlichkeiten sammeln
top_words = [(word, predictions[index]) for word, index in tokenizer.word_index.items() if index in top_indices]

# Sortiere die Wörter nach Wahrscheinlichkeit in absteigender Reihenfolge
top_words = sorted(top_words, key=lambda x: x[1], reverse=True)

# Ausgabe der Top-10 Wörter und Wahrscheinlichkeiten
print(f"\nTop {top_k} nächste Wörter für '{seed_text}':")
for word, prob in top_words:
    print(f"{word}: {(prob * 100):.4f} %")


Eingabe: 'Ich' tokeiziert zu: [37]
Mit Paddig: [[ 0  0  0  0  0  0  0  0  0  0  0 37]]

Top 10 nächste Wörter für 'Ich':
bin: 19.1827 %
hoffe: 18.0123 %
liebe: 17.0989 %
muss: 14.0489 %
fehle: 12.0559 %
predige: 9.1918 %
stelle: 7.4289 %
wird: 0.0741 %
wir: 0.0687 %
war: 0.0612 %


## Konvertierung 

In [94]:
if convert_to_js_model:
    print(f"Konveriert neues Modell in '{js_model_dir}'\n")
    ! tensorflowjs_converter --input_format=tf_saved_model {model_file} {js_model_dir}

    print(f"Kopiert Tokenizer in '{js_model_dir}'")
    ! cp -r {word_index_file} {js_model_dir}
    
    ! cd {model_dir}
    ! ls -al {model_dir}

else:
    print("Konvertiertes Model bereits vorhanden")


Konveriert neues Modell in '/workspace/models/hero_to_zero_lstm/lstm_js'

2024-11-12 12:48:56.194209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 12:48:56.194522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 12:48:56.218777: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 12:48:58.458279: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Kopiert Tokenizer in '/workspace/models/hero_to_zero_lstm/lstm_js'
total 2028
drwxr-xr-x 1 root root    4096 Nov 12 12:47 .
drwxr-xr-x 1 root root    4096 Nov 12 12:49

## In React Kopieren 

In [95]:
if copie_js_model_to_react:
    print("Kopiere JS-Model in React-Model")
    ! cp -r {js_model_dir} {react_model_dir}
else:
    print("JS-Model bereits im React-Model vorhanden")

Kopiere JS-Model in React-Model
